In [9]:
%pip install seaborn
%pip install matplotlib

You should consider upgrading via the '/Users/ame/.pyenv/versions/3.9.6/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/Users/ame/.pyenv/versions/3.9.6/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [10]:
from keras.models import Sequential
import keras
from tensorflow.keras.layers import Conv2D, Flatten
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dense

import matplotlib.pyplot as plt
%matplotlib inline 

In [11]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
gpus

if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)


In [12]:
#Model
# CONSTANTS
imagesize = 128
OUTPUT = 7
CLASSES = ['HeadShotHyalomma', 'HeadShotRhipicephalus',
        'CenterShotHyalomma', 'CenterShotRhipicephalus',
        'CornerShotHyalomma', 'CornerShotRhipicephalus',
        'Unidentified']

model = Sequential()
model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(imagesize, imagesize, 3)))
model.add(Conv2D(16, (3,3), 1, activation='relu'))
model.add(AveragePooling2D(pool_size=(2,2)))
model.add(Conv2D(16, (3,3), 1, activation='relu'))
model.add(Conv2D(128, (3,3), 1, activation='relu'))
model.add(AveragePooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(OUTPUT, activation='softmax'))

model.summary()

/Users/ame/.pyenv/versions/3.9.6/lib/python3.9/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)               │ (None, 126, 126, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 124, 124, 16)   │         2,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_2             │ (None, 62, 62, 16)     │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 60, 60, 16)     │         2,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 58, 58, 128)    │        18,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_3             │ (None, 29, 29, 128)    │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 107648)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 7)              │       753,543 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 777,191 (2.96 MB)

 Trainable params: 777,191 (2.96 MB)

 Non-trainable params: 0 (0.00 B)

In [13]:
DENSE_LAYER_ACTIVATION = 'softmax'
OBJECTIVE_FUNCTION = 'categorical_crossentropy'
LOSS_METRICS = ['accuracy']
CLASS_MODE = 'categorical'
NUM_EPOCHS = 20

In [14]:
from keras.optimizers import Adam, SGD
opt = Adam(learning_rate=0.0001)
model.compile(optimizer=opt, loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

In [15]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# preprocessing_function is applied on each image but only after re-sizing & augmentation (resize => augment => pre-process)
# Each of the keras.application.resnet* preprocess_input MOSTLY mean BATCH NORMALIZATION (applied on each batch) stabilize the inputs to nonlinear activation functions
# Batch Normalization helps in faster convergence
# data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)
data_generator = ImageDataGenerator(rescale=1.0/255.0)

# flow_From_directory generates batches of augmented data (where augmentation can be color conversion, etc)
# Both train & valid folders must have NUM_CLASSES sub-folders
train_generator = data_generator.flow_from_directory(
        'datasetB/train',
        target_size=(imagesize, imagesize),
        # batch_size=BATCH_SIZE_TRAINING,
        class_mode=CLASS_MODE,
        classes=CLASSES)

validation_generator = data_generator.flow_from_directory(
        'datasetB/dev',
        target_size=(imagesize, imagesize),
        # batch_size=BATCH_SIZE_VALIDATION,
        class_mode=CLASS_MODE,
        classes=CLASSES
) 

Found 14574 images belonging to 7 classes.
Found 6071 images belonging to 7 classes.


In [16]:
fit_history = model.fit(
        train_generator,
        epochs = NUM_EPOCHS,
        validation_data=validation_generator,     
)

Epoch 1/20


/Users/ame/.pyenv/versions/3.9.6/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


456/456 ━━━━━━━━━━━━━━━━━━━━ 441s 950ms/step - accuracy: 0.4463 - loss: 1.4515 - val_accuracy: 0.5551 - val_loss: 1.1054
Epoch 2/20
456/456 ━━━━━━━━━━━━━━━━━━━━ 457s 994ms/step - accuracy: 0.6418 - loss: 0.9540 - val_accuracy: 0.6808 - val_loss: 0.8514
Epoch 3/20
456/456 ━━━━━━━━━━━━━━━━━━━━ 469s 1s/step - accuracy: 0.6972 - loss: 0.8077 - val_accuracy: 0.7114 - val_loss: 0.8068
Epoch 4/20
456/456 ━━━━━━━━━━━━━━━━━━━━ 458s 994ms/step - accuracy: 0.7569 - loss: 0.6880 - val_accuracy: 0.7472 - val_loss: 0.7103
Epoch 5/20
456/456 ━━━━━━━━━━━━━━━━━━━━ 455s 989ms/step - accuracy: 0.7879 - loss: 0.6088 - val_accuracy: 0.7572 - val_loss: 0.6769
Epoch 6/20
456/456 ━━━━━━━━━━━━━━━━━━━━ 461s 1s/step - accuracy: 0.8208 - loss: 0.5093 - val_accuracy: 0.7648 - val_loss: 0.6421
Epoch 7/20
456/456 ━━━━━━━━━━━━━━━━━━━━ 467s 1s/step - accuracy: 0.8419 - loss: 0.4708 - val_accuracy: 0.7925 - val_loss: 0.5850
Epoch 8/20
456/456 ━━━━━━━━━━━━━━━━━━━━ 468s 1s/step - accuracy: 0.8571 - loss: 0.4223 - val_acc

In [17]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, classification_report, roc_auc_score, roc_curve

model_type = "CNN"
variant = "VARB"
count = "00k"
epochs = NUM_EPOCHS
result_folder_name = "cnn"
results_folder = f'results/b/{result_folder_name}/{str(NUM_EPOCHS)}'


# Create results folder if it doesn't exist
if not os.path.exists(results_folder):
    os.makedirs(results_folder)

In [18]:
plt.figure(1, figsize = (15,8)) 
    
# plt.subplot(221)  
plt.plot(fit_history.history['accuracy'])  
plt.plot(fit_history.history['val_accuracy'])  
plt.title('model accuracy')  
plt.ylabel('accuracy')  
plt.xlabel('epoch')  
plt.legend(['train', 'valid']) 
plt.savefig(os.path.join(results_folder, 'model-accuracy.png'))
plt.close()

# plt.subplot(222)  
plt.plot(fit_history.history['loss'])  
plt.plot(fit_history.history['val_loss'])  
plt.title('model loss')  
plt.ylabel('loss')  
plt.xlabel('epoch')  
plt.legend(['train', 'valid']) 
plt.savefig(os.path.join(results_folder, 'model-loss.png'))
plt.close()

In [19]:
# Make predictions on the test set
training_loss, training_accuracy = model.evaluate(train_generator)
print("Training Loss:", training_loss)
print("Training Accuracy:", training_accuracy)

validation_loss, validation_accuracy = model.evaluate(train_generator)
print("Validation Loss:", validation_loss)
print("Validation Accuracy:", validation_accuracy)

456/456 ━━━━━━━━━━━━━━━━━━━━ 235s 515ms/step - accuracy: 0.9802 - loss: 0.0779
Training Loss: 0.07390842586755753
Training Accuracy: 0.9815424680709839
456/456 ━━━━━━━━━━━━━━━━━━━━ 252s 553ms/step - accuracy: 0.9814 - loss: 0.0747
Validation Loss: 0.07390842586755753
Validation Accuracy: 0.9815424680709839


In [20]:
# Save ROC AUC
with open(os.path.join(results_folder, 'training_validation.txt'), 'w') as f:
    f.write(f'training_accuracy: {training_accuracy}\n')
    f.write(f'training_loss: {training_loss}\n')
    f.write(f'validation_accuracy: {validation_accuracy}\n')
    f.write(f'validation_loss: {validation_loss}\n')

In [21]:
# # Define the model loading function
# def load_my_model(model_path):
#     return load_model(model_path)

# Paths and constants
# model_path = 'models/' + name
data_folder = 'datasetB/test'

# Load the model
# model = load_my_model(model_path)

# Data generator for the test set
testingData = ImageDataGenerator(rescale=1.0/255.0)

testdata = testingData.flow_from_directory(
    data_folder,
    target_size=(imagesize, imagesize),  # Change this to the input size of your model
    class_mode='categorical',  # or 'binary' depending on your model
    shuffle=False,
    classes=CLASSES
)



Found 5165 images belonging to 7 classes.


In [22]:
# Make predictions on the test set
test_loss, test_accuracy = model.evaluate(testdata)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

with open(os.path.join(results_folder, 'test.txt'), 'w') as f:
    f.write(f'test_accuracy: {test_accuracy}\n')
    f.write(f'test_loss: {test_loss}\n')

y_true = testdata.classes
y_pred_probs = model.predict(testdata, steps=None)

print("testdata.samples:", testdata.samples)
print("len(y_true):", len(y_true))

y_pred = np.argmax(y_pred_probs, axis=1)
print("y_pred: ", len(y_pred))

/Users/ame/.pyenv/versions/3.9.6/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


162/162 ━━━━━━━━━━━━━━━━━━━━ 70s 427ms/step - accuracy: 0.8587 - loss: 0.5991
Test Loss: 0.4509667158126831
Test Accuracy: 0.8880929350852966
162/162 ━━━━━━━━━━━━━━━━━━━━ 58s 359ms/step
testdata.samples: 5165
len(y_true): 5165
y_pred:  5165


In [23]:
# Calculate confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)

# Calculate accuracy
accuracy = accuracy_score(y_true, y_pred)

# Calculate precision
precision = precision_score(y_true, y_pred, average='weighted')

# Calculate sensitivity (recall)
sensitivity = recall_score(y_true, y_pred, average='weighted')

# Calculate F1 score
f1 = precision_score(y_true, y_pred, average='weighted')

# Print the results
print("Confusion Matrix:")
print(conf_matrix)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Sensitivity (Recall):", sensitivity)
print("F1 Score:", f1)

# Calculate classification report
report = classification_report(y_true, y_pred, target_names=CLASSES, output_dict=True)

# Save classification report
report_df = pd.DataFrame(report).transpose()
report_df.to_csv(os.path.join(results_folder, 'classification_report.csv'))

# Calculate ROC AUC score
roc_auc = roc_auc_score(y_true, y_pred_probs, multi_class='ovo')

# Save ROC AUC
with open(os.path.join(results_folder, 'roc_auc.txt'), 'w') as f:
    f.write(f'ROC AUC: {roc_auc}\n')

# Calculate sensitivity and specificity for each class
sensitivity_per_class = np.diag(conf_matrix) / np.sum(conf_matrix, axis=1)

# Calculate true negatives, false positives, and specificity for each class
tn = []
fp = []
specificity_per_class = []

for i in range(len(CLASSES)):
    true_negative = np.sum(conf_matrix) - (np.sum(conf_matrix[i, :]) + np.sum(conf_matrix[:, i]) - conf_matrix[i, i])
    false_positive = np.sum(conf_matrix[:, i]) - conf_matrix[i, i]
    tn.append(true_negative)
    fp.append(false_positive)
    specificity_per_class.append(true_negative / (true_negative + false_positive))

# Save confusion matrix, sensitivity, specificity, and recall
conf_matrix_df = pd.DataFrame(conf_matrix, index=CLASSES, columns=CLASSES)
conf_matrix_df.to_csv(os.path.join(results_folder, 'confusion_matrix.csv'))

sensitivity_df = pd.DataFrame([sensitivity_per_class], index=['Sensitivity'], columns=CLASSES)
sensitivity_df.to_csv(os.path.join(results_folder, 'sensitivity.csv'))

specificity_df = pd.DataFrame([specificity_per_class], index=['Specificity'], columns=CLASSES)
specificity_df.to_csv(os.path.join(results_folder, 'specificity.csv'))

recall_df = pd.DataFrame([sensitivity_per_class], index=['Recall'], columns=CLASSES)
recall_df.to_csv(os.path.join(results_folder, 'recall.csv'))

# Plot and save confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=CLASSES, yticklabels=CLASSES)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.savefig(os.path.join(results_folder, 'confusion_matrix.png'))
plt.close()

# Plot and save ROC curve
plt.figure()
for i, class_name in enumerate(CLASSES):
    fpr, tpr, _ = roc_curve(y_true == i, y_pred_probs[:, i])
    plt.plot(fpr, tpr, label=f'{class_name} (AUC: {roc_auc_score(y_true == i, y_pred_probs[:, i]):.2f})')

plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.savefig(os.path.join(results_folder, 'roc_curve.png'))
plt.close()

print("Results and plots saved to", results_folder)

Confusion Matrix:
[[ 156    2    1    0    8    3    7]
 [   3  429    1   10    0   19   17]
 [   2    1  262    7   17    7    1]
 [   0   26   12  448    1   69   19]
 [  15    3   24    4  171   15   12]
 [   2   45   11   66   10  577   23]
 [  12   26   12   16    9   40 2544]]
Accuracy: 0.8880929332042594
Precision: 0.8891104507551854
Sensitivity (Recall): 0.8880929332042594
F1 Score: 0.8891104507551854
Results and plots saved to results/b/cnn/20


In [24]:
from datetime import date
import math

accuracy = accuracy * 100 
accuracy = math.floor(accuracy * 100) / 100

today = date.today()

# dd/mm/YY
date = "-".join(today.strftime("%d/%m/%Y").split("/"))
print(date)

name = f"{model_type}_{variant}{count}_{imagesize}x{imagesize}_EP{epochs}_ACCU{accuracy}_{date}.keras"

model.save('models/' + name)

24-06-2024
